In [ ]:
! pip install fedimpute==0.1.0
! pip install tabulate matplotlib numpy pandas

In [1]:
cd ..

d:\research\FedImpute


d:\research\FedImpute\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import numpy as np
import pandas as pd
import tabulate

# Load Data

In [8]:
%load_ext autoreload
%autoreload 2

from fedimpute.data_prep import load_data, display_data, column_check
from fedimpute.scenario import ScenarioBuilder
data, data_config = load_data("fed_heart_disease")
scenario_builder = ScenarioBuilder()
scenario_data = scenario_builder.create_real_scenario(
    data, data_config,
)
scenario_builder.summarize_scenario()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Scenario Summary
Total clients: 4
Global Test Data: (94, 21)
Missing Mechanism Category: MCAR
Clients Data Summary:
     Train     Test      Miss     MS Ratio    MS Feature    Seed
--  --------  -------  --------  ----------  ------------  ------
C1  (244,21)  (28,21)  (244,20)     0.00         0/20       6077
C2  (237,21)  (27,21)  (237,20)     0.04         4/20       577
C3  (99,21)   (11,21)  (99,20)      0.01         3/20       7231
C4  (162,21)  (18,21)  (162,20)     0.07         5/20       5504



In [9]:
scenario_builder.visualize_missing_pattern(
    client_ids=[0, 1, 2, 3], data_type='train', fontsize=20, save_path='./plots/real_pattern_train.png'
)

In [10]:
scenario_builder.visualize_missing_pattern(
    client_ids=[0, 1, 2, 3], data_type='test', fontsize=20, save_path='./plots/real_pattern_test.png'
)

# Running Federated Imputation

In [11]:
%load_ext autoreload
%autoreload 2
from fedimpute.execution_environment import FedImputeEnv

env = FedImputeEnv(debug_mode=False)
env.configuration(imputer = 'mice', fed_strategy='fedmice', workflow_params = {})
env.setup_from_scenario_builder(scenario_builder = scenario_builder, verbose=1)
env.show_env_info()
env.run_fed_imputation(verbose=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Setting up clients...
Setting up server...
Setting up workflow...
Environment setup complete.
Environment Information:
Workflow: ICE (Imputation via Chain Equation)
Clients:
 - Client 0: imputer: mice, fed-strategy: fedmice
 - Client 1: imputer: mice, fed-strategy: fedmice
 - Client 2: imputer: mice, fed-strategy: fedmice
 - Client 3: imputer: mice, fed-strategy: fedmice
Server: fed-strategy: fedmice

Imputation Start ...


ICE Iterations:   0%|          | 0/20 [00:00<?, ?it/s]

Feature_idx:   0%|          | 0/20 [00:00<?, ?it/s]

Feature_idx:   0%|          | 0/20 [00:00<?, ?it/s]

Feature_idx:   0%|          | 0/20 [00:00<?, ?it/s]

...

Feature_idx:   0%|          | 0/20 [00:00<?, ?it/s]

Feature_idx:   0%|          | 0/20 [00:00<?, ?it/s]

Feature_idx:   0%|          | 0/20 [00:00<?, ?it/s]

Feature_idx:   0%|          | 0/20 [00:00<?, ?it/s]

Finished. Running time: 0.9958 seconds


# Evaluation

In [14]:
%load_ext autoreload
%autoreload 2
from fedimpute.evaluation import Evaluator

evaluator = Evaluator()

X_trains, y_trains = env.get_data(client_ids='all', data_type = 'train', include_y=True)
X_tests, y_tests = env.get_data(client_ids='all', data_type = 'test', include_y=True)
X_train_imps = env.get_data(client_ids='all', data_type = 'train_imp')
X_test_imps = env.get_data(client_ids='all', data_type = 'test_imp')
data_config = env.get_data(data_type = 'config')

ret = evaluator.run_local_prediction(
    X_train_imps = X_train_imps,
    y_trains = y_trains,
    X_tests = X_test_imps,
    y_tests = y_tests,
    data_config = data_config,
    model = 'lr',
    seed= 0
)
evaluator.show_local_prediction_results()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Clients:   0%|          | 0/4 [00:00<?, ?it/s]

Downstream Prediction (Local)
             accuracy       f1         auc         prc
----------  ----------  ----------  ----------  ----------
 Client 1     0.750       0.741       0.831       0.853
 Client 2     0.889       0.842       0.947       0.931
 Client 3     0.818       0.900       0.000       0.798
 Client 4     0.500       0.571       0.662       0.777
----------  ----------  ----------  ----------  ----------
 Average      0.739       0.764       0.610       0.839
   Std        0.147       0.125       0.366       0.059


### Federated Prediction

In [16]:
X_train_imps = env.get_data(client_ids='all', data_type = 'train_imp')
X_test_imps = env.get_data(client_ids='all', data_type = 'test_imp')
X_global_test_imp = env.get_data(data_type = 'global_test_imp')

_ , y_trains = env.get_data(
    client_ids='all', data_type = 'train', include_y=True
)
_ , y_tests = env.get_data(
    client_ids='all', data_type = 'test', include_y=True
)
_, y_global_test = env.get_data(
    data_type = 'global_test', include_y = True
)
data_config = env.get_data(data_type = 'config')

ret = evaluator.run_fed_prediction(
    X_train_imps = X_train_imps,
    y_trains = y_trains,
    X_tests = X_test_imps,
    y_tests = y_tests,
    X_test_global = X_global_test_imp,
    y_test_global = y_global_test,
    data_config = data_config,
    model_name = 'lr',
    seed= 0
)

evaluator.show_fed_prediction_results()

(149, 20) (149,)


                                                                3.02it/s]

Downstream Prediction (Fed)
 Personalized    accuracy       f1         auc         prc
--------------  ----------  ----------  ----------  ----------
   Client 1       0.643       0.667       0.749       0.779
   Client 2       0.889       0.824       0.982       0.970
   Client 3       0.545       0.706       0.100       0.848
   Client 4       0.556       0.636       0.477       0.708
  ----------    ----------  ----------  ----------  ----------
    Global        0.819       0.817       0.902       0.919
